In [21]:
import pandas as pd
import numpy as np
import os

### Eerst gaan we alle datafiles die we nodig hebben mergen tot 1 dataframe

#### 1. Alles van Account mergen

##### Account mergen met activiteit tabellen

In [2]:
data_acc = pd.read_csv('../data_clean/Account_fixed.csv', sep=',')
data_acti = pd.read_csv('../data_clean/Activiteitscode_fixed.csv', sep=",")
data_acc_acti = pd.read_csv('../data_clean/Account_activiteitscode_fixed.csv', sep=",")

# merge account_activiteitscode with activiteitscode
merged_relation = data_acc_acti.merge(data_acti, left_on='account_activiteitscode_activiteitscode', right_on='activiteitscode_activiteitscode_id', how='inner')
# merge account with the merged relation from above
merged_accounts = data_acc.merge(merged_relation, left_on='account_account_id', right_on='account_activiteitscode_account', how='inner')

In [3]:
# RAM vrijmaken
try:
    del data_acc
    del data_acti
    del data_acc_acti
    del merged_relation
except:
    print("Data is niet meer beschikbaar / is al verwijderd")

In [4]:
merged_accounts.shape

(14673, 22)

##### Accounts mergen met financiële data

In [5]:
financiele_data = pd.read_csv('../data_clean/Account_financiële_data_fixed.csv', sep=',')
financiele_data.shape

(2030180, 6)

In [6]:
merged_accounts = merged_accounts.merge(financiele_data, left_on='account_account_id', right_on='financieledata_ondernemingid', how='inner')
merged_accounts.shape

(181428, 28)

##### Accounts merge met Contactfiches

In [7]:
contactfiches = pd.read_csv('../data_clean/Contact_fixed.csv', sep=',')
contactfiches.shape

(1169683, 6)

In [8]:
merged_accounts = merged_accounts.merge(contactfiches, left_on='account_account_id', right_on='contact_account', how='inner')
merged_accounts.shape

(3748445, 34)

##### Accounts mergen met Functie

In [9]:
contactfiches_functies = pd.read_csv('../data_clean/Contact_functie_fixed.csv', sep=',')
functies = pd.read_csv('../data_clean/Functie_fixed.csv', sep=',')
contactfiches_functies.shape, functies.shape

((589129, 2), (71, 2))

In [10]:
contactfiches_functies.columns

Index(['contactfunctie_contactpersoon', 'contactfunctie_functie'], dtype='object')

In [11]:
merged_accounts = merged_accounts.merge(contactfiches_functies, left_on='contact_contactpersoon_id', right_on='contactfunctie_contactpersoon', how='inner')

In [12]:
list(merged_accounts.columns)

['account_account_id',
 'account_adres_geografische_regio',
 'account_adres_geografische_subregio',
 'account_adres_plaats',
 'account_adres_postcode',
 'account_adres_provincie',
 'account_industriezone_naam_',
 'account_is_voka_entiteit',
 'account_ondernemingsaard',
 'account_ondernemingstype',
 'account_oprichtingsdatum',
 'account_primaire_activiteit',
 'account_reden_van_status',
 'account_status',
 'account_voka_nr_',
 'account_adres_land',
 'account_activiteitscode_account',
 'account_activiteitscode_activiteitscode',
 'account_activiteitscode_inf_account_inf_activiteitscodeid',
 'activiteitscode_naam',
 'activiteitscode_activiteitscode_id',
 'activiteitscode_status',
 'financieledata_ondernemingid',
 'financieledata_boekjaar',
 'financieledata_aantal_maanden',
 'financieledata_toegevoegde_waarde',
 'financieledata_fte',
 'financieledata_gewijzigd_op',
 'contact_contactpersoon_id',
 'contact_account',
 'contact_functietitel',
 'contact_persoon_id',
 'contact_status',
 'contact_

In [13]:
merged_accounts = merged_accounts.merge(functies, left_on='contactfunctie_functie', right_on='functie_functie_id', how='inner')

##### Accounts merge met Personen

In [14]:
personen = pd.read_csv('../data_clean/Persoon_fixed.csv', sep=',')
personen.shape

(580981, 29)

In [15]:
merged_accounts = merged_accounts.merge(personen, left_on='contact_persoon_id', right_on='persoon_persoon_id', how='inner')
merged_accounts.shape

(1771670, 67)

#### 2. Alles van Afspraken mergen

In [42]:
afspraak_betreft_account_cleaned = pd.read_csv('../data_clean/Afspraak_betreft_account_cleaned_fixed.csv', sep=',')
afspraak_account_gelinkt_cleaned = pd.read_csv('../data_clean/Afspraak_account_gelinkt_cleaned_fixed.csv', sep=',')
afspraak_betreft_contact_cleaned = pd.read_csv('../data_clean/Afspraak_betreft_contact_cleaned_fixed.csv', sep=',')

In [43]:
afspraak_betreft_account_cleaned.shape, afspraak_account_gelinkt_cleaned.shape, afspraak_betreft_contact_cleaned.shape

((4876, 7), (2934, 7), (2552, 7))

In [44]:
afspraak_betreft_account_cleaned = afspraak_betreft_account_cleaned.drop(['afspraak_betreft_account_betreft_id'], axis=1)
afspraak_account_gelinkt_cleaned = afspraak_account_gelinkt_cleaned.drop(['afspraak_account_gelinkt_account'], axis=1)
afspraak_betreft_contact_cleaned = afspraak_betreft_contact_cleaned.drop(['afspraak_betreft_contactfiche_betreft_id'], axis=1)

In [45]:
list(afspraak_betreft_account_cleaned.columns)

['afspraak_betreft_account_afspraak_id',
 'afspraak_betreft_account_thema',
 'afspraak_betreft_account_subthema',
 'afspraak_betreft_account_onderwerp',
 'afspraak_betreft_account_eindtijd',
 'afspraak_betreft_account_keyphrases']

In [46]:
list(afspraak_account_gelinkt_cleaned.columns)

['afspraak_account_gelinkt_afspraak_id',
 'afspraak_account_gelinkt_thema',
 'afspraak_account_gelinkt_subthema',
 'afspraak_account_gelinkt_onderwerp',
 'afspraak_account_gelinkt_eindtijd',
 'afspraak_account_gelinkt_keyphrases']

In [47]:
list(afspraak_betreft_contact_cleaned.columns)

['afspraak_betreft_contactfiche_afspraak_id',
 'afspraak_betreft_contactfiche_thema',
 'afspraak_betreft_contactfiche_subthema',
 'afspraak_betreft_contactfiche_onderwerp',
 'afspraak_betreft_contactfiche_eindtijd',
 'afspraak_betreft_contactfiche_keyphrases']

In [48]:
afspraak_betreft_account_cleaned.columns = ['afspraak_afspraak_id', 'afspraak_thema', 'afspraak_subthema', 'afspraak_onderwerp', 'afspraak_eindtijd', 'afspraak_keyphrases']
afspraak_account_gelinkt_cleaned.columns = ['afspraak_afspraak_id', 'afspraak_thema', 'afspraak_subthema', 'afspraak_onderwerp', 'afspraak_eindtijd', 'afspraak_keyphrases']
afspraak_betreft_contact_cleaned.columns = ['afspraak_afspraak_id', 'afspraak_thema', 'afspraak_subthema', 'afspraak_onderwerp', 'afspraak_eindtijd', 'afspraak_keyphrases']

In [49]:
afspraak_betreft_account_cleaned["afspraak_betreft"] = "account"
afspraak_account_gelinkt_cleaned["afspraak_betreft"] = "account"
afspraak_betreft_contact_cleaned["afspraak_betreft"] = "contact"

In [50]:
afspraken_merged = pd.concat([afspraak_betreft_account_cleaned, afspraak_account_gelinkt_cleaned, afspraak_betreft_contact_cleaned])

In [51]:
afspraken = pd.read_csv('../data_clean/Afspraak_alle_fixed.csv', sep=',')
afspraken.columns = ['afspraak_afspraak_id']
afspraken.head()

,afspraak_afspraak_id
0,00082DAA-24BF-ED11-83FF-6045BD8952CE
1,00109C77-4D09-EB11-8114-001DD8B72B61
2,0011A5D9-CC9A-EC11-B400-0022488005A7
3,0016532E-14E3-EC11-BB3C-000D3ABCA38B
4,003760C1-13B4-EC11-983F-00224883C880


In [52]:
afspraken_merged = pd.merge(afspraken, afspraken_merged, on='afspraak_afspraak_id', how='right')
afspraken_merged.reset_index(drop=True, inplace=True)

In [53]:
afspraken_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10362 entries, 0 to 10361
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   afspraak_afspraak_id  10362 non-null  object
 1   afspraak_thema        10362 non-null  object
 2   afspraak_subthema     10362 non-null  object
 3   afspraak_onderwerp    10362 non-null  object
 4   afspraak_eindtijd     10362 non-null  object
 5   afspraak_keyphrases   10362 non-null  object
 6   afspraak_betreft      10362 non-null  object
dtypes: object(7)
memory usage: 566.8+ KB


In [54]:
# RAM vrijmaken
try:
    del afspraak_betreft_account_cleaned
    del afspraak_account_gelinkt_cleaned
    del afspraak_betreft_contact_cleaned
    del afspraken
except:
    print("Data is niet meer beschikbaar / is al verwijderd")

In [55]:
afspraken_merged = afspraken_merged.drop_duplicates(subset=['afspraak_afspraak_id'], keep='first')
afspraken_merged.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7624 entries, 0 to 10357
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   afspraak_afspraak_id  7624 non-null   object
 1   afspraak_thema        7624 non-null   object
 2   afspraak_subthema     7624 non-null   object
 3   afspraak_onderwerp    7624 non-null   object
 4   afspraak_eindtijd     7624 non-null   object
 5   afspraak_keyphrases   7624 non-null   object
 6   afspraak_betreft      7624 non-null   object
dtypes: object(7)
memory usage: 476.5+ KB


In [56]:
activiteit_vereist_contact = pd.read_csv('../data_clean/Activiteit_vereist_contact_fixed.csv', sep=',')
activiteit_vereist_contact.shape

(4635, 2)

In [57]:
afspraken_merged = afspraken_merged.merge(activiteit_vereist_contact, left_on='afspraak_afspraak_id', right_on='activiteitvereistcontact_activityid_id', how='left')
afspraken_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7754 entries, 0 to 7753
Data columns (total 9 columns):
 #   Column                                  Non-Null Count  Dtype 
---  ------                                  --------------  ----- 
 0   afspraak_afspraak_id                    7754 non-null   object
 1   afspraak_thema                          7754 non-null   object
 2   afspraak_subthema                       7754 non-null   object
 3   afspraak_onderwerp                      7754 non-null   object
 4   afspraak_eindtijd                       7754 non-null   object
 5   afspraak_keyphrases                     7754 non-null   object
 6   afspraak_betreft                        7754 non-null   object
 7   activiteitvereistcontact_activityid_id  4545 non-null   object
 8   activiteitvereistcontact_reqattendee    4545 non-null   object
dtypes: object(9)
memory usage: 545.3+ KB


#### 3. Alles van Campagnes mergen

##### Campagne mergen met Sessie

In [59]:
campagne = pd.read_csv('../data_clean/Campagne_fixed.csv', sep=',')
campagne.shape

(168, 11)

In [58]:
sessie = pd.read_csv('../data_clean/Sessie_fixed.csv', sep=',')
sessie.shape

(350, 8)

In [60]:
campagnes_merged = campagne.merge(sessie, left_on='campagne_campagne_id', right_on='sessie_campagne', how='inner')
campagnes_merged.shape

(350, 19)

##### Campagnes mergen met Sessie inschrijving

In [61]:
sessie_inschrijving = pd.read_csv('../data_clean/Sessie_inschrijving_fixed.csv', sep=',')
sessie_inschrijving.shape

(10159, 3)

In [62]:
campagnes_merged = campagnes_merged.merge(sessie_inschrijving, left_on='sessie_sessie_id', right_on='sessieinschrijving_sessie', how='inner')
campagnes_merged.shape

(10159, 22)

#### 4. Inschrijving omzetten naar 1 kolom, de target feature

#### 5. Alles mergen tot 1 dataframe

### Data cleaning / One hot encoding

##### Op voorhand onnodige kolommen verwijderen voor minder geheugengebruik

In [29]:
onnodige_kolommen = [
 'account_adres_postcode',
 'account_industriezone_naam_',
 'account_oprichtingsdatum',
 'account_voka_nr_',
 'account_activiteitscode_account',
 'account_activiteitscode_activiteitscode',
 'account_activiteitscode_inf_account_inf_activiteitscodeid',
 'activiteitscode_activiteitscode_id',
 'afspraak_betreft_account_afspraak_id',
 'afspraak_betreft_account_onderwerp',
 'afspraak_betreft_account_betreft_id',
 'afspraak_betreft_account_eindtijd',
 'afspraak_betreft_account_keyphrases',
 'afspraak_account_gelinkt_afspraak_id',
 'afspraak_account_gelinkt_onderwerp',
 'afspraak_account_gelinkt_eindtijd',
 'afspraak_account_gelinkt_account',
 'afspraak_account_gelinkt_keyphrases',
 'contact_account',
 'contact_functietitel',
 'contact_persoon_id',
 'afspraak_betreft_contactfiche_afspraak_id',
 'afspraak_betreft_contactfiche_onderwerp',
 'afspraak_betreft_contactfiche_betreft_id',
 'afspraak_betreft_contactfiche_eindtijd',
 'afspraak_betreft_contactfiche_keyphrases',
 'persoon_persoon_id',
 'persoon_persoonsnr_', 
 'campagne_campagne_id',
 'campagne_campagne_nr',
 'campagne_einddatum',
 'campagne_naam',
 'campagne_naam_in_email',
 'campagne_startdatum',
 'campagne_url_voka_be', 
 'sessie_campagne',
 'sessie_eind_datum_tijd',
 'sessie_sessie_id',
 'sessie_sessie_nr_',
 'sessie_start_datum_tijd',
]

(0, 66)